In [4]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat


from hloc import extract_features, match_features, localize_inloc, visualization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pipeline for indoor localization

## Setup
Here we declare the paths to the dataset, image pairs, and we choose the feature extractor and the matcher. You need to download the [InLoc dataset](https://www.visuallocalization.net/datasets/) and put it in `datasets/inloc/`, or change the path.

In [6]:
dataset = Path('../datasets/inloc_small/')  # change this if your dataset is somewhere else

pairs = Path('../pairs/inloc_small/')
loc_pairs = pairs / 'pairs-query-netvlad40-custom-shub-small.txt'  # top 40 retrieved by NetVLAD

outputs = Path('../outputs/inloc_small/')  # where everything will be saved
results = outputs / 'InLoc_hloc_superpoint+superglue_netvlad40.txt'  # the result file

In [7]:
# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

Configs for feature extractors:
{'d2net-ss': {'model': {'multiscale': False, 'name': 'd2net'},
              'output': 'feats-d2net-ss',
              'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'dir': {'model': {'name': 'dir'},
         'output': 'global-feats-dir',
         'preprocessing': {'resize_max': 1024}},
 'netvlad': {'model': {'name': 'netvlad'},
             'output': 'global-feats-netvlad',
             'preprocessing': {'resize_max': 1024}},
 'sift': {'model': {'name': 'sift'},
          'output': 'feats-sift',
          'preprocessing': {'grayscale': True, 'resize_max': 1600}},
 'superpoint_aachen': {'model': {'max_keypoints': 4096,
                                 'name': 'superpoint',
                                 'nms_radius': 3},
                       'output': 'feats-superpoint-n4096-r1024',
                       'preprocessing': {'grayscale': True,
                                         'resize_max': 1024}},
 'superpoint_inloc': {'model': {'

In [8]:
# pick one of the configurations for extraction and matching
# you can also simply write your own here!
feature_conf = extract_features.confs['superpoint_inloc']
matcher_conf = match_features.confs['superglue']

## Extract local features for database and query images

In [9]:
feature_path = extract_features.main(feature_conf, dataset, outputs)

[09/23/2021 18:20:34 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[09/23/2021 18:20:35 INFO] Found 3956 images in root ../datasets/inloc_small.
Loaded SuperPoint model


 45%|████▌     | 1798/3956 [00:51<00:56, 38.03it/s]/home/shubodh/anaconda3/envs/hloc/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
 60%|██████    | 2386/3956 [03:05<02:02, 12.84it/s]
Traceback (most recent call last):
  File "/home/shubodh/anaconda3/envs/hloc/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/shubodh/anaconda3/envs/hloc/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/shubodh/anaconda3/envs/hloc/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
  

KeyboardInterrupt: 

## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [ ]:
match_path = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. Different from when localizing with Aachen, here we do not need a 3D SfM model here: the dataset already has 3D lidar scans. The file `InLoc_hloc_superpoint+superglue_netvlad40.txt` will contain the estimated query poses.

In [ ]:
localize_inloc.main(
    dataset, loc_pairs, feature_path, match_path, results,
    skip_matches=20)  # skip database images with too few matches

## Visualization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [ ]:
visualization.visualize_loc(results, dataset, n=1, top_k_db=1, seed=2)